# PyTorch MNIST example dissected

In this notebook we'll explore the components of the
[PyTorch MNIST example](https://github.com/pytorch/examples/tree/master/mnist)
one-by-one.

**This is notebook 2 of the multi-part example.** Please see [Part 1 here](1_mnist_load.ipynb).

## 2. Building the model

We'll need just one import in this notebook:

In [1]:
import torch